![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import re  # regular expressions for work with str data
from datetime import datetime, timedelta
from collections import Counter

import requests as req
from lxml import html

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [138]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [139]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [140]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [141]:
df_train.info()

In [142]:
df_train.head(5)

In [143]:
df_test.info()

In [144]:
df_test.head(5)

In [145]:
sample_submission.head(5)

In [146]:
sample_submission.info()

In [147]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [148]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [149]:
data.sample(5)

In [150]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [151]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [152]:
data['Number_of_Reviews_isNAN']

In [153]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

## 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [154]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

### 'City'

In [155]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [156]:
data.head(5)

In [157]:
data.sample(5)

### 'Price Range'
Возьмем следующий признак "Price Range".

In [158]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [159]:
# Обработка 'Price Range':
# 1. Создаем словарь для перевода значений в числовой тип данных
# 2. Считаем медианное значение и заполняем им пропуски
# 3. Приводим данные к целочисленному типу

dicr_for_transform_price = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
data['Price Range'] = data['Price Range'].map(dicr_for_transform_price)
data['Price Range'] = (data['Price Range'].fillna(data['Price Range'].median())).apply(int)

data['Price Range'].sample(5)

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### My functions

In [160]:
def checking_column(column):
    # Showing the general information
    print(column.describe())
    
    # Showing the head
    print(f'\nHead of data: \n{column.head()}\n')
    
    # Showing unique values
    print(f"A slice of unique values:\n{column.value_counts()[:6]}\n"
          f"The number of repetitions and the number of unique values (with such a number of repetitions)\n"
          f"{Counter(column.value_counts()).most_common(20)}")

## 'Restaurant_id'
Restaurant_id — идентификационный номер ресторана / сети ресторанов 

In [161]:
checking_column(data['Restaurant_id'])

In [162]:
# Transform to int (Removing string characters and converting to an integer)
data['Restaurant_id'] = [i[3:] for i in data['Restaurant_id']]
data['Restaurant_id'] = data['Restaurant_id'].apply(int)


checking_column(data['Restaurant_id'])
# Restaurants can be divided into groups according to the size of the chain.
# Let's divide it into 4 groups: 
# 1. single_restaurant - only 1 restaurant (not a chain)
# 2. small_restaurant_group - number of restaurants from 2 to 7
# 3. medium_restaurant_group - number of restaurants from 8 to 12
# 4. big_restaurant_group - number of restaurants more than 13

In [163]:
# Create new features in Dataset
new_features_chain_size = dict(data['Restaurant_id'].value_counts())

data['single_restaurant'] = data['Restaurant_id'].apply(lambda x: 1 if new_features_chain_size[x] == 1 else 0)
data['small_restaurant_chain'] = data['Restaurant_id'].apply(lambda x: 1 if 1 < new_features_chain_size[x] < 8 else 0)
data['medium_restaurant_chain'] = data['Restaurant_id'].apply(lambda x: 1 if 7 < new_features_chain_size[x] < 13 else 0)
data['large_restaurant_chain'] = data['Restaurant_id'].apply(lambda x: 1 if new_features_chain_size[x] > 12 else 0)

data.sample(5)

## Cuisine Style
Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане 

In [164]:
def get_restorant_cuisines(cuisines_one_rest):
    # Selecting from the list of cuisines of one restaurant (the order of transformations is important)
    # Check None
    if pd.notna(cuisines_one_rest):
        cuisines_one_rest = cuisines_one_rest[1:-1].replace(" ", "").replace("'", " ").replace(" ,", "").split()
    
        return cuisines_one_rest
    
    # Fill omissions (na)
    else:
        return ['Unknown']
    
    
def create_cuisines_set(df_cuisines):
    # Use it before transform a cuisines feature to list type 
    set_cuisines = set()
    df_cuisines = df_cuisines.apply(set)
    for row_cuisines in df_cuisines:
        set_cuisines = set_cuisines|row_cuisines
    
    return set_cuisines


def create_dummy_cuisines(df, all_cuisines):
    shape = df.shape[0]
    for cuisine in all_cuisines:
        df[cuisine] = np.zeros(shape, dtype=int)
    
    return df


def fill_cuisines(df_row):
    for cuisine in df_row['Cuisine Style']:
        df_row[cuisine] = 1
    
    return df_row

In [165]:
checking_column(data['Cuisine Style']) 

# Plan:
# 0. Transform str of cuisines to list of ones
# 1. Create to a set with cuisines 
# 2. Create to dummies features for cuisines 
# 3. Fill the dummies features for each restorant 
# 4. Count the number of cuisines of each restorant and write it into cuisines_number feature
# 5. Check my changes 

In [166]:
# 0. Transform str of cuisines to list of ones
data['Cuisine Style'] = data['Cuisine Style'].apply(get_restorant_cuisines)

# 1. Create to a set with cuisines 
all_cuisines = create_cuisines_set(data['Cuisine Style'])

# 2. Create to dummies features for cuisines 
data = create_dummy_cuisines(data, all_cuisines)

# 3. Fill the dummies features for each restorant 
data = data.apply(fill_cuisines, axis=1)

# 4. Count the number of cuisines of each restorant and write it into cuisines_number feature
data['Cuisine Style'] = data['Cuisine Style'].apply(len)

# 5. Check my changes 
print(data.head())
print("\n", data['Cuisine Style'].value_counts())

### Ranking
Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города ('ranking')

In [167]:
checking_column(data['Ranking'])
data['Ranking'].unique()
# Only numerical data in this feature, there are no omissions. Logically, this parameter should be an integer

In [168]:
# Check float values
data['Ranking'].apply(lambda x: x%1).unique()
# No number has a decimal part -> transform to int type

In [169]:
data['Ranking'] = data['Ranking'].apply(int)

# Check result
checking_column(data['Ranking'])

## Number of Reviews 
Number of Reviews — количество отзывов о ресторане

In [170]:
checking_column(data['Number of Reviews'])

In [171]:
# There are omissions -> fill its to the median velue
data['Number of Reviews'] = data['Number of Reviews'].fillna(data['Number of Reviews'].median())

In [172]:
# Logically, the data should be of an integer type, check it
print(data['Number of Reviews'].apply(lambda x: x%1).unique())

# Converte to int type 
data['Number of Reviews'] = data['Number of Reviews'].apply(int)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [173]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [174]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [175]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [176]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [177]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [178]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [179]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [180]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [181]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [182]:
def get_restorant_cuisines(cuisines_one_rest):
    # Selecting from the list of cuisines of one restaurant (the order of transformations is important)
    # Check None
    if pd.notna(cuisines_one_rest):
        cuisines_one_rest = cuisines_one_rest[1:-1].replace(" ", "").replace("'", " ").replace(" ,", "").split()
    
        return cuisines_one_rest
    
    # Fill omissions (na)
    else:
        return ['Unknown']
    
    
def create_cuisines_set(df_cuisines):
    # Use it before transform a cuisines feature to list type 
    set_cuisines = set()
    df_cuisines = df_cuisines.apply(set)
    for row_cuisines in df_cuisines:
        set_cuisines = set_cuisines|row_cuisines
    
    return set_cuisines


def create_dummy_cuisines(df, all_cuisines):
    shape = df.shape[0]
    for cuisine in all_cuisines:
        df[cuisine] = np.zeros(shape, dtype=int)
    
    return df


def fill_cuisines(df_row):
    for cuisine in df_row['Cuisine Style']:
        df_row[cuisine] = 1
    
    return df_row


def transform_price_range_to_numbers(df_origin_price):
    price = df_origin_price.copy()
    
    # Replace symbols with numbers
    dicr_for_transform_price = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
    price = price.map(dicr_for_transform_price)

    return price    


def fillna_price_range(df_origin_price):
    price = df_origin_price.copy()
    
    # Fill na taking into account the preservation of the proportion of restaurants among themselves in terms of quantity
    distr = price.value_counts()
    
    count_of_nan = data.shape[0] - distr.sum()
    # Remmember: '$': 1, '$$ - $$$': 2, '$$$$': 3
    count_1 = int((distr[1] / distr.sum()) * count_of_nan)
    count_3 = int((distr[3] / distr.sum()) * count_of_nan)
    count_2 = count_of_nan - count_1 - count_3

    series_for_fillna = pd.Series(np.ones(count_1)).append(pd.Series(np.ones(count_2)*2)).append(pd.Series(np.ones(count_3)*3))
    series_for_fillna.index = price[price.isna() == True].index

    price = price.fillna(series_for_fillna)
    
    return price


def fillna_count_reviws(df_origin):
    df = df_origin.copy()
    
    series_for_fillna = pd.Series()
    
    all_cities = re.findall('(City_\w+)', str(df.columns))
    for city in all_cities:
        number_reviews_for_this_city = df[df[city] == 1]['Number of Reviews']
        median_for_this_city = number_reviews_for_this_city.median()
        nan_indexes = number_reviews_for_this_city[number_reviews_for_this_city.isna() == True].index
        
        df.loc[nan_indexes, 'Number of Reviews'] = median_for_this_city
        
#         series_for_fillna = series_for_fillna.append(pd.Series(median_for_this_city, index=nan_indexes))
        
#     df['Number of Reviews'].fillna(series_for_fillna, inplace=True)
#     print('series_for_fillna\n',series_for_fillna)
    
    return df


def create_review_dates(df):
    df['Reviews'] = df['Reviews'].fillna('[[], []]')
    
    pattern = re.compile('\'\d+\/\d+\/\d+\'')
    dates = df['Reviews'].apply(pattern.findall)
    # врeменные признаки
    df['date1'] = pd.to_datetime(dates.apply(lambda x: x[0] if len(x) > 0 else None))
    df['date2'] = pd.to_datetime(dates.apply(lambda x: x[1] if len(x) > 1 else None))


    return df


def days_since_last_review(row):
    # Считаем количество дней с последней даты отзыва, учитывая отсутствие одной из дат и пропуск 
    datetime_now = datetime.now();
    if pd.notna(row.date1) and pd.notna(row.date2):
        if row.date1 > row.date2:
            return (datetime_now - row.date1).days
        else:
            return (datetime_now - row.date2).days
    else:
        if pd.notna(row.date1):
            return (datetime_now - row.date1).days
        if pd.notna(row.date2):
            return (datetime_now - row.date2).days        
    return 0 


def reviews_days_delta(row):
    # Считаем количество дней между двумя отзывами, учитывая отсутствие (одной из) дат 
    if pd.notna(row.date1) and pd.notna(row.date2):
        return abs((row.date1 - row.date2).days)
    else:
        return 0


In [184]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['ID_TA',], axis = 1, inplace=True)

    
    # ################### 'Restaurant_id' ###############################################   
    # Убираем лишние начальные символы и преобразуем в целочисленный формат
    df_output['Restaurant_id'] = [i[3:] for i in df_output['Restaurant_id']]
    df_output['Restaurant_id'] = df_output['Restaurant_id'].apply(int)
    
    # Создаю новые категориальные признаки основываясь на столбце 'Restaurant_id':
    # Размер сети ресторана:
    # single_restaurant - только один ресторан 
    # small_restaurant_chain - от 2 до 7 рестаранов в сети
    # medium_restaurant_chain - от 8 до 13 рестаранов в сети
    # large_restaurant_chain - более 13 рестаранов в сети
    
    # Создаю словарь для заполнения новых признаков числами
    new_features_chain_size = dict(df_output['Restaurant_id'].value_counts())
    
    # Создаю новые признаки и сразу заполняю их значениями
    df_output['single_restaurant'] = df_output['Restaurant_id'].apply(lambda x: 1 if new_features_chain_size[x] == 1 else 0)
    df_output['small_restaurant_chain'] = df_output['Restaurant_id'].apply(lambda x: 1 if 1 < new_features_chain_size[x] <= 7 else 0)
    df_output['medium_restaurant_chain'] = df_output['Restaurant_id'].apply(lambda x: 1 if 7 < new_features_chain_size[x] <= 13 else 0)
    df_output['large_restaurant_chain'] = df_output['Restaurant_id'].apply(lambda x: 1 if new_features_chain_size[x] > 13 else 0)

     
    # ################### 'City' ########################################################   
    # Создаю dummies признаки для городов
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
        
    # ################### 'Cuisine Style' ###############################################
    # Разбиваю строку с кухнями на список, обрабатывая немного (пробельные символы и т.д.)
    df_output['Cuisine Style'] = df_output['Cuisine Style'].apply(get_restorant_cuisines)

    # Создаю множество со всеми кухнями  
    all_cuisines = create_cuisines_set(df_output['Cuisine Style'])

    # Создаю для каждой кухний свой столбец  
    df_output = create_dummy_cuisines(df_output, all_cuisines)

    # Заполняю для каждого ресторана его кухни  
    df_output = df_output.apply(fill_cuisines, axis=1)

    # Добавляю признак 'Number Cuisines' - число кухонь у 1 ресторана 
    df_output['Number Cuisines'] = df_output['Cuisine Style'].apply(len)
    
    
    # ################### 'Ranking' #####################################################
    # Место в городе среди ресторанов это обязаельно целое число
    df_output['Ranking'] = df_output['Ranking'].apply(int)
    
    
    # ################### 'Rating' ######################################################
    # Пропусков нет, тип данных соответствующий, это целевой параметр предсказания.    
    
    
    # ################### 'Price Range' #################################################
    # Преобразую '$' в 1, '$$-$$$' в 2,  '$$$$' в 3
    df_output['Price Range'] = transform_price_range_to_numbers(df_output['Price Range'])
    
    # Заполняю пропуски с соъранением соотношения между кол-вами ресторанов между собой
    df_output['Price Range'] = fillna_price_range(df_output['Price Range'])
    
    # ################### 'Number of Reviews' ###########################################
    # Заполняю пропуски количества отзывов средним среди ресторанов города данного ресторана 
    df_output['Number of Reviews'] = df_output['Number of Reviews'].fillna(df_output['Number of Reviews'].median())
    
    # Количество отзывов - это всегда целое число 
    df_output['Number of Reviews'] = df_output['Number of Reviews'].apply(int)
       
    
    # ################### 'Reviews' #####################################################
    # Создадим 2 новых параметра: 
    # 1. Количество дней между двумя последними отзывами
    # 2. Количество дней со времени последнего отзыва
    # Создаем вспомогательные признаки:
    df_output = create_review_dates(df_output)
    # Создаем новые параметры
    df_output['Days_Since_Review'] = df_output.apply(lambda row: days_since_last_review(row), axis=1)
    df_output['Reviews_Days_Delta'] = df_output.apply(lambda row: reviews_days_delta(row), axis=1)
    # Удаляем временные признаки
    df_output.drop(['date1', 'date2'], inplace=True, axis=1)
    
    # ################### 'URL_TA' ######################################################  
    # Данный признак не использовался в системе
    
    # ################### 'ID_TA' #######################################################  
    #  Данный признак не использовался в системе
        
    # ################### Clean #########################################################
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)

    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

In [185]:
df = data.copy()
df = preproc_data(df)

In [186]:
df.info()

#### Запускаем и проверяем что получилось

In [187]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [188]:
df_preproc.info()

In [189]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [190]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [191]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [192]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [193]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [194]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [195]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [196]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [197]:
test_data.sample(10)

In [198]:
test_data = test_data.drop(['Rating'], axis=1)

In [199]:
sample_submission

In [200]:
predict_submission = model.predict(test_data)

In [201]:
predict_submission

In [202]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
